In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [3]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [5]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, pickup_location_hex_8
from orders.order_logs_snapshot
where yyyymmdd >= '20230505'
and yyyymmdd <= '20230511'
and city_name in ('Chennai')
and service_obj_service_name = 'Link'
and hour in ('12', '13', '14', '15', '16')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [6]:
#Create a copy of the df retrived from presto
df = df_copy.copy()

In [7]:
#View the dataset
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,pickup_location_hex_8
0,645767d798037a2035d960d8,1.034,88618c4c67fffff,13.044573,80.193015,Chennai,88618c4c67fffff,13.044412,80.192881,142655,14,1415,Link,14:00,7,20230507,customerCancelled,Change of plans,88618c4c67fffff
1,645767e6d50bd7069e12a9ac,0.737,88618c4883fffff,13.077084,80.260559,Chennai,88618c4d61fffff,13.088343,80.226646,142710,14,1415,Link,14:00,7,20230507,dropped,None,88618c4d61fffff
2,645769ab0b1f5803c31486ee,0.721,88618c4d33fffff,13.095252,80.185692,Chennai,88618c4d33fffff,13.095250,80.185638,143443,14,1430,Link,14:59,7,20230507,customerCancelled,Change of plans,88618c4d33fffff
3,64576a7d14f02e74a3db5018,2.430,88618c4f65fffff,13.042031,80.259186,Chennai,88618c4f65fffff,13.042178,80.259094,143813,14,1430,Link,14:59,7,20230507,dropped,None,88618c4f65fffff
4,64576a7fa105c02a77c2ac8f,2.476,88618c471dfffff,12.858436,80.185478,Chennai,88618c473dfffff,12.881905,80.192375,143815,14,1430,Link,14:59,7,20230507,customerCancelled,Change of plans,88618c473dfffff


In [8]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,pickup_location_hex_8,second,minute,date
0,645767d798037a2035d960d8,1.034,88618c4c67fffff,13.044573,80.193015,Chennai,88618c4c67fffff,13.044412,80.192881,142655,14,1415,Link,14:00,7,20230507,customerCancelled,Change of plans,88618c4c67fffff,55,26,07
1,645767e6d50bd7069e12a9ac,0.737,88618c4883fffff,13.077084,80.260559,Chennai,88618c4d61fffff,13.088343,80.226646,142710,14,1415,Link,14:00,7,20230507,dropped,None,88618c4d61fffff,10,27,07
2,645769ab0b1f5803c31486ee,0.721,88618c4d33fffff,13.095252,80.185692,Chennai,88618c4d33fffff,13.095250,80.185638,143443,14,1430,Link,14:59,7,20230507,customerCancelled,Change of plans,88618c4d33fffff,43,34,07
3,64576a7d14f02e74a3db5018,2.430,88618c4f65fffff,13.042031,80.259186,Chennai,88618c4f65fffff,13.042178,80.259094,143813,14,1430,Link,14:59,7,20230507,dropped,None,88618c4f65fffff,13,38,07
4,64576a7fa105c02a77c2ac8f,2.476,88618c471dfffff,12.858436,80.185478,Chennai,88618c473dfffff,12.881905,80.192375,143815,14,1430,Link,14:59,7,20230507,customerCancelled,Change of plans,88618c473dfffff,15,38,07


In [9]:
#Create 15-sec batches and 20-sec batches
df['second'] = df['second'].astype(float)
df['minute'] = df['minute'].astype(float)
df['20_sec_batch'] = pd.cut(df['second'], bins = [-1, 20, 40, 61], labels = ['0-20', '20-40', '40-60'])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,pickup_location_hex_8,second,minute,date,20_sec_batch,count
0,645767d798037a2035d960d8,1.034,88618c4c67fffff,13.044573,80.193015,Chennai,88618c4c67fffff,13.044412,80.192881,142655,14,1415,Link,14:00,7,20230507,customerCancelled,Change of plans,88618c4c67fffff,55.0,26.0,07,40-60,1
1,645767e6d50bd7069e12a9ac,0.737,88618c4883fffff,13.077084,80.260559,Chennai,88618c4d61fffff,13.088343,80.226646,142710,14,1415,Link,14:00,7,20230507,dropped,None,88618c4d61fffff,10.0,27.0,07,0-20,1
2,645769ab0b1f5803c31486ee,0.721,88618c4d33fffff,13.095252,80.185692,Chennai,88618c4d33fffff,13.095250,80.185638,143443,14,1430,Link,14:59,7,20230507,customerCancelled,Change of plans,88618c4d33fffff,43.0,34.0,07,40-60,1
3,64576a7d14f02e74a3db5018,2.430,88618c4f65fffff,13.042031,80.259186,Chennai,88618c4f65fffff,13.042178,80.259094,143813,14,1430,Link,14:59,7,20230507,dropped,None,88618c4f65fffff,13.0,38.0,07,0-20,1
4,64576a7fa105c02a77c2ac8f,2.476,88618c471dfffff,12.858436,80.185478,Chennai,88618c473dfffff,12.881905,80.192375,143815,14,1430,Link,14:59,7,20230507,customerCancelled,Change of plans,88618c473dfffff,15.0,38.0,07,0-20,1


In [10]:
#Check the number of rides coming from each hex
df_grouped = df.groupby(['pickup_location_hex_8'])['count'].sum().reset_index()

#Filter out those hexes where atleast 1000 rides have come in the last two weeks
df_grouped = df_grouped[df_grouped['count'] >= 100].reset_index(drop = True)

#Add a column to keep for analysis
df_grouped['keep_for_analysis'] = 'yes'
df_grouped.head()

,pickup_location_hex_8,count,keep_for_analysis
0,88618c4081fffff,129,yes
1,88618c4087fffff,171,yes
2,88618c408bfffff,305,yes
3,88618c4099fffff,144,yes
4,88618c409bfffff,141,yes


In [13]:
#Merge the number of rides to the original df
df = df.merge(df_grouped[['pickup_location_hex_8', 'keep_for_analysis']], on = 'pickup_location_hex_8', how = 'left')
df['keep_for_analysis'] = df['keep_for_analysis'].fillna('no')
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,pickup_location_hex_8,second,minute,date,20_sec_batch,count,keep_for_analysis
0,645767d798037a2035d960d8,1.034,88618c4c67fffff,13.044573,80.193015,Chennai,88618c4c67fffff,13.044412,80.192881,142655,14,1415,Link,14:00,7,20230507,customerCancelled,Change of plans,88618c4c67fffff,55.0,26.0,07,40-60,1,yes
1,645767e6d50bd7069e12a9ac,0.737,88618c4883fffff,13.077084,80.260559,Chennai,88618c4d61fffff,13.088343,80.226646,142710,14,1415,Link,14:00,7,20230507,dropped,None,88618c4d61fffff,10.0,27.0,07,0-20,1,yes
2,645769ab0b1f5803c31486ee,0.721,88618c4d33fffff,13.095252,80.185692,Chennai,88618c4d33fffff,13.095250,80.185638,143443,14,1430,Link,14:59,7,20230507,customerCancelled,Change of plans,88618c4d33fffff,43.0,34.0,07,40-60,1,yes
3,64576a7d14f02e74a3db5018,2.430,88618c4f65fffff,13.042031,80.259186,Chennai,88618c4f65fffff,13.042178,80.259094,143813,14,1430,Link,14:59,7,20230507,dropped,None,88618c4f65fffff,13.0,38.0,07,0-20,1,yes
4,64576a7fa105c02a77c2ac8f,2.476,88618c471dfffff,12.858436,80.185478,Chennai,88618c473dfffff,12.881905,80.192375,143815,14,1430,Link,14:59,7,20230507,customerCancelled,Change of plans,88618c473dfffff,15.0,38.0,07,0-20,1,no


In [14]:
#Filter the df only on hexes to keep
df_filter = df[df['keep_for_analysis'] == 'yes']

In [15]:
#Groupby on hex to see number of orders coming in 15 and 20 second batches
df_hex_grouped = df_filter.groupby(['pickup_location_hex_8', 'date', 'hour', 'minute', '20_sec_batch'])['count'].sum().reset_index()
df_hex_grouped = df_hex_grouped.sort_values(by = ['pickup_location_hex_8', 'date', 'hour', 'minute', '20_sec_batch'])
df_hex_grouped.head()

,pickup_location_hex_8,date,hour,minute,20_sec_batch,count
0,88618c4081fffff,05,12,0.0,0-20,0
1,88618c4081fffff,05,12,0.0,20-40,0
2,88618c4081fffff,05,12,0.0,40-60,0
3,88618c4081fffff,05,12,1.0,0-20,0
4,88618c4081fffff,05,12,1.0,20-40,0


In [16]:
#Identify high demand hexes
df_hex_grouped_filter2 = df_hex_grouped[df_hex_grouped['count'] >= 1]
count_df = pd.DataFrame(df_hex_grouped_filter2['pickup_location_hex_8'].value_counts().reset_index())
count_df.columns = ['pickup_location_hex_8', 'count']
count_df.head(20)

,pickup_location_hex_8,count
0,88618c4885fffff,1533
1,88618c7307fffff,1400
2,88618c4f1bfffff,1270
3,88618c4883fffff,1260
4,88618c4c05fffff,1233
5,88618c4889fffff,1196
6,88618c4d0bfffff,1169
7,88618c4d5dfffff,1130
8,88618c48d5fffff,1105
9,88618c48a9fffff,964


In [17]:
df[['pickup_location_hex_8', 'count']].to_csv('chennai_hexes.csv', index = False)

In [18]:
#Look at the average demand for each minute
df_hex_grouped['time'] = (pd.to_datetime(df_hex_grouped['hour'].astype(str) + ':' + df_hex_grouped['minute'].astype(int).astype(str), format='%H:%M').dt.time)
df_hex_grouped.head()

,pickup_location_hex_8,date,hour,minute,20_sec_batch,count,time
0,88618c4081fffff,05,12,0.0,0-20,0,12:00:00
1,88618c4081fffff,05,12,0.0,20-40,0,12:00:00
2,88618c4081fffff,05,12,0.0,40-60,0,12:00:00
3,88618c4081fffff,05,12,1.0,0-20,0,12:01:00
4,88618c4081fffff,05,12,1.0,20-40,0,12:01:00


In [14]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Hyderabad')
and resolution = 8
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [15]:
hexes = df_cluster_copy[df_cluster_copy['cluster'].isin(['Nagole', 'Dilsukhnagar', 'Victoria Memorial Metro', 'Saroor Nagar', 'Saroor Nagar Lake Park',
 'shilparamam', 'Bahadurguda', 'Shamshabad', 'kharmanghat', 'L B Nagar Metro'])]['hex_id'].unique().tolist()

In [19]:
#Filter on the hexes identified as high demand
high_demand_hexes = ['88618c48e7fffff', '88618c48a9fffff', '88618c4885fffff', '88618c48b9fffff', '88618c4883fffff', '88618c4889fffff', '88618c48d5fffff',
 '88618c48c3fffff', '88618c48d3fffff', '88618c4f25fffff']
hexes = []
for x in high_demand_hexes:
    hex_ids = h3.k_ring(x, 1)
    hexes.append(hex_ids)

hexes = [item for sublist in hexes for item in sublist]

In [25]:
print(len(hexes))

70


In [24]:
pd.DataFrame(hexes).to_csv('afternoon_chennai_zone.csv', index = False)

In [22]:
#Get the total number of rides per hour
df_hex_high_demand = df_hex_grouped[df_hex_grouped['pickup_location_hex_8'].isin(hexes)]
df_level_1_grouped = round(df_hex_high_demand.groupby(['date', 'hour'])['count'].sum().reset_index(), 0)
df_level_1_grouped = df_level_1_grouped.rename(columns = {'count':'rides_per_hour'})
df_level_1_grouped['rides_per_20_sec'] = df_level_1_grouped['rides_per_hour'].apply(lambda x: round(x/180, 2))
df_level_1_grouped

,date,hour,rides_per_hour,rides_per_20_sec
0,05,12,731,4.06
1,05,13,825,4.58
2,05,14,837,4.65
3,05,15,800,4.44
4,05,16,827,4.59
5,06,12,800,4.44
6,06,13,866,4.81
7,06,14,920,5.11
8,06,15,859,4.77
9,06,16,821,4.56


In [21]:
jaipur_hexes = df_hex_high_demand['customer_location_hex_8'].unique().tolist()
print(jaipur_hexes)

['883da20a61fffff', '883da20a65fffff', '883da20a69fffff', '883da20a6dfffff', '883da20b59fffff', '883da21837fffff', '883da218a1fffff', '883da218a5fffff', '883da218a7fffff', '883da218adfffff', '883da21981fffff', '883da21983fffff', '883da21987fffff', '883da21989fffff', '883da2198bfffff', '883da2198dfffff', '883da21991fffff', '883da21995fffff', '883da21997fffff', '883da21999fffff', '883da2199bfffff', '883da2199dfffff', '883da219b1fffff', '883da219b3fffff', '883da219b7fffff', '883da219b9fffff', '883da219bbfffff', '883da219bdfffff', '883da219c1fffff', '883da219c3fffff', '883da219c5fffff', '883da219c7fffff', '883da219cbfffff', '883da219d1fffff', '883da219d3fffff', '883da219d5fffff', '883da219d7fffff', '883da219d9fffff', '883da219dbfffff', '883da219ddfffff', '883da219ebfffff']


In [22]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Jaipur')
and resolution = 8
and hex_id in ('883da20a61fffff', '883da20a65fffff', '883da20a69fffff', '883da20a6dfffff', '883da20b59fffff', '883da21837fffff', '883da218a1fffff',
 '883da218a5fffff', '883da218a7fffff', '883da218adfffff', '883da21981fffff', '883da21983fffff', '883da21987fffff', '883da21989fffff', '883da2198bfffff',
  '883da2198dfffff', '883da21991fffff', '883da21995fffff', '883da21997fffff', '883da21999fffff', '883da2199bfffff', '883da2199dfffff', '883da219b1fffff',
   '883da219b3fffff', '883da219b7fffff', '883da219b9fffff', '883da219bbfffff', '883da219bdfffff', '883da219c1fffff', '883da219c3fffff', '883da219c5fffff',
    '883da219c7fffff', '883da219cbfffff', '883da219d1fffff', '883da219d3fffff', '883da219d5fffff', '883da219d7fffff', '883da219d9fffff', '883da219dbfffff',
     '883da219ddfffff', '883da219ebfffff')
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [23]:
df_cluster_copy['cluster'].value_counts()

Gopalpura_JAI                 8
Malviya_Nagar_JAI 2           5
Mansarovar 2_JAI              3
Nirman_Nagar_JAI              2
Sudarshanpura_JAI             2
Malviya Nagar_JAI             2
Maharani Farm_JAI             2
MNIT_JAI                      2
Durgapura_JAI                 2
                              2
Swej Farm_JAI                 2
Satkar_Malviya_Nagar_JAI      1
Triveni_Nagar_JAI             1
VT Road Mansarovar_JAI        1
Jhalana Dungari_JAI           1
Lalkothi_JAI                  1
Narayan Vihar_JAI             1
Landscape_Park_JAI            1
Swarna Path Mansarovar_JAI    1
Gurjar_Ki_Thadi_JAI           1
Name: cluster, dtype: int64

In [17]:
df_level_1_grouped[df_level_1_grouped['hour'].isin(['08', '09', '10', '11', '17', '18', '19', '20', '21'])]

,date,hour,rides_per_hour,rides_per_20_sec
8,06,08,2214,12.30
9,06,09,1811,10.06
10,06,10,1406,7.81
11,06,11,1155,6.42
17,06,17,1351,7.51
18,06,18,1409,7.83
19,06,19,1331,7.39
20,06,20,1177,6.54
21,06,21,953,5.29
32,07,08,1429,7.94


In [18]:
df_level_1_grouped.groupby(['hour'])['rides_per_20_sec'].median()

hour
00    0.38
01    0.23
02    0.17
03    0.16
04    0.73
05    1.46
06    1.02
07    1.38
08    2.18
09    4.96
10    4.12
11    3.29
12    3.76
13    3.79
14    3.56
15    3.57
16    3.56
17    3.73
18    4.57
19    4.21
20    3.13
21    2.37
22    1.59
23    0.76
Name: rides_per_20_sec, dtype: float64

In [19]:
df_level_1_grouped.groupby(['hour'])['rides_per_20_sec'].mean()

hour
00    0.388
01    0.224
02    0.170
03    0.152
04    0.656
05    1.504
06    1.020
07    1.366
08    2.252
09    4.652
10    4.164
11    3.356
12    3.716
13    3.892
14    3.974
15    3.574
16    3.478
17    3.590
18    4.510
19    4.254
20    3.348
21    2.420
22    1.714
23    0.766
Name: rides_per_20_sec, dtype: float64

In [44]:
df_hex_high_demand_grouped.to_csv('demand_chennai.csv', index = False)

In [43]:
df_hex_high_demand_grouped = df_hex_high_demand.groupby(['customer_location_hex_8'])['count'].sum().reset_index()
df_hex_high_demand_grouped

,customer_location_hex_8,count
0,88618c4881fffff,1095
1,88618c4883fffff,3407
2,88618c4885fffff,3705
3,88618c4887fffff,1339
4,88618c4889fffff,2186
5,88618c488bfffff,1321
6,88618c488dfffff,369
7,88618c4891fffff,1183
8,88618c4893fffff,816
9,88618c4895fffff,1343
